In [16]:
from pulp import *
import pulp
import pandas as pd
import numpy as np

In [17]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
products = []

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1'}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2'}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3'}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4'}]


In [18]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [19]:
productTest = calculate_profit(products)
print(productTest)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]


In [20]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [21]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

[{'p1': p1}, {'p2': p2}, {'p3': p3}, {'p4': p4}]


In [22]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [23]:
#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

Please insert minimum total amount of products: 10
Please insert maximum  total amount of products: 50
Please insert minimum  amount of products : 1
Please insert maximum  amount of products : 10
Please insert minimum number of product variety: 1
Please insert maximum  number of product variety: 4
Please insert store dimension: 200000000000


In [24]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
model += variety <= maxProductVariety
model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 


4
10
1
5*p1 + 2*p2 + p3 + 20*p4


In [25]:
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
SUBJECT TO
_C1: p1 + p2 + p3 + p4 <= 50

_C2: p1 + p2 + p3 + p4 >= 10

_C3: p1 <= 10

_C4: p1 >= 1

_C5: p2 <= 10

_C6: p2 >= 1

_C7: p3 <= 10

_C8: p3 >= 1

_C9: p4 <= 10

_C10: p4 >= 1

_C11: 5 p1 + 2 p2 + p3 + 20 p4 <= 200000000000

VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [26]:
# Solve the problem
status = model.solve()

In [27]:
print(pulp.LpStatus[status])

Optimal


In [28]:
#print(pulp.value(p1), pulp.value(model.objective))

In [29]:
# Print the solution
#print("Solution:")
#print("x =", x.value())
#print("y =", y.value())

In [30]:
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()

p1 = 10
p2 = 10
p3 = 10
p4 = 10


In [31]:
import random

# Define the fitness function
def fitness(x):
    # x is the number of products to buy
    # We can define the fitness as the difference between the desired stock level and the actual stock level
    # The desired stock level is 10
    return abs(10 - x)

# Initialize the population with random solutions
population = [random.randint(0, 20) for _ in range(10)]

# Set the maximum number of generations
max_generations = 100

# Iterate over the generations
for generation in range(max_generations):
    # Evaluate the fitness of each solution
    fitness_values = [fitness(x) for x in population]
    
    # Select the best solutions as parents
    # We'll use tournament selection here, which involves selecting a random subset of the population
    # and choosing the best solution from that subset as the parent
    parent1 = min(random.sample(population, 5), key=fitness)
    parent2 = min(random.sample(population, 5), key=fitness)
    
    # Create offspring by combining the parents using crossover
    # We'll use one-point crossover here, which involves selecting a random point in the parents' genomes
    # and swapping the genes on either side of the point to create the offspring
    crossover_point = random.randint(1, len(parent1))
    offspring = parent1[:crossover_point] + parent2[crossover_point:]
    
    # Add some randomness to the offspring using mutation
    # We'll flip a random bit in the offspring's genome
    mutation_point = random.randint(0, len(offspring) - 1)
    offspring[mutation_point] = 1 - offspring[mutation_point]
    
    # Replace the worst solution in the population with the offspring
    worst_solution = max(zip(population, fitness_values), key=lambda x: x[1])
    population[population.index(worst_solution[0])] = offspring

# The final solution is the best solution in the final generation
best_solution = min(zip(population, fitness_values), key=lambda x: x[1])
print(f"The optimal number of products to buy is: {best_solution[0]}")

TypeError: object of type 'int' has no len()

In [ ]:
def maximize_profit(product_quantities, product_dimensions, store_dimension, min_quantity, max_quantity, target_profit):
  # evaluate the fitness of a given solution
  def evaluate_fitness(solution):
    total_quantity = sum(solution)
    total_dimension = sum([q * d for q, d in zip(solution, product_dimensions)])
    if total_quantity < min_quantity or total_quantity > max_quantity or total_dimension > store_dimension:
      return 0
    profit = calculate_profit(solution)
    # calculate the deviation from the target profit
    deviation = abs(profit - target_profit)
    # the fitness is higher for solutions that have a lower deviation from the target profit
    return 1 / (deviation + 1)

  # genetic algorithm function
  def genetic_algorithm(fitness_fn):
    # initialize population
    population = initialize_population(product_quantities)
    for generation in range(max_generations):
      # evaluate fitness
      fitnesses = [fitness_fn(s) for s in population]
      # select parents
      parents = select_parents(population, fitnesses)
      # create offspring
      offspring = create_offspring(parents)
      # mutate offspring
      offspring = mutate(offspring)
      # replace least fit individuals in population with offspring
      population = replace_least_fit(population, offspring)
    # return the fittest solution
    return population[0]

  # use the genetic algorithm to find the optimal solution
  return genetic_algorithm(evaluate_fitness)